In [6]:
import numpy as np

In [12]:
# Parameters
np.random.seed(0)  # for reproducibility
num_states = 5
num_actions = 30
gamma = 0.9

# Random transition probabilities P[s, a, s'] — rows must sum to 1
P = np.zeros((num_states, num_actions, num_states))
for s in range(num_states):
    for a in range(num_actions):
        probs = np.random.rand(num_states)
        probs /= probs.sum()  # normalize to sum to 1
        P[s, a] = probs

# Random rewards R[s, a] in range [-10, 10]
R = np.random.uniform(-10, 10, (num_states, num_actions))

In [35]:
actionsub=[16,14,15,25,8]

In [36]:
# Initialize policy arbitrarily
policy = np.random.choice(num_actions, size=num_states)

def policy_evaluation(policy, P, R, gamma, tol=1e-6):
    V = np.zeros(num_states)
    while True:
        delta = 0
        for s in range(num_states):
            a = policy[s]
            v = V[s]
            V[s] = R[s, a] + gamma * np.dot(P[s, a], V)
            delta = max(delta, abs(v - V[s]))
        if delta < tol:
            break
    return V

def policy_improvement(V, P, R, gamma):
    policy_stable = True
    new_policy = np.zeros(num_states, dtype=int)
    for s in range(num_states):
        old_action = policy[s]
        action_values = [
            R[s, a] + gamma * np.dot(P[s, a], V) for a in actionsub
        ]
        new_policy[s] = actionsub[np.argmax(action_values)]
        if new_policy[s] != old_action:
            policy_stable = False
    return new_policy, policy_stable

# Policy Iteration
iteration = 0
while True:
    print(f"Iteration {iteration}: Policy = {policy}")
    V = policy_evaluation(policy, P, R, gamma)
    policy, stable = policy_improvement(V, P, R, gamma)
    if stable:
        break
    iteration += 1

# Final results
np.set_printoptions(precision=3, suppress=True)
print("\nOptimal Policy:", policy)
print("Value Function:", V)
# print("\nTransition Probabilities P (shape {}):".format(P.shape))
# print(P)
# print("\nReward Matrix R (shape {}):".format(R.shape))
# print(R)


Iteration 0: Policy = [ 1 22  3 16  3]
Iteration 1: Policy = [14 14 15 25  8]
Iteration 2: Policy = [16 14 15 25  8]

Optimal Policy: [16 14 15 25  8]
Value Function: [91.661 91.558 90.363 90.807 90.22 ]


In [49]:
# Initialize policy arbitrarily
policy = np.random.choice(num_actions, size=num_states)

def policy_evaluation(policy, P, R, gamma, tol=1e-6):
    V = np.zeros(num_states)
    while True:
        delta = 0
        for s in range(num_states):
            a = policy[s]
            v = V[s]
            V[s] = R[s, a] + gamma * np.dot(P[s, a], V)
            delta = max(delta, abs(v - V[s]))
        if delta < tol:
            break
    return V

def policy_improvement(V, P, R, gamma):
    policy_stable = True
    new_policy = np.zeros(num_states, dtype=int)
    for s in range(num_states):
        old_action = policy[s]
        action_values = [
            R[s, a] + gamma * np.dot(P[s, a], V) for a in range(num_actions)
        ]
        new_policy[s] = np.argmax(action_values)
        if new_policy[s] != old_action:
            policy_stable = False
    return new_policy, policy_stable

# Policy Iteration
iteration = 0
while True:
    print(f"Iteration {iteration}: Policy = {policy}")
    V = policy_evaluation(policy, P, R, gamma)
    policy, stable = policy_improvement(V, P, R, gamma)
    if stable:
        break
    iteration += 1

# Final results
np.set_printoptions(precision=3, suppress=True)
print("\nOptimal Policy:", policy)
print("Value Function:", V)
# print("\nTransition Probabilities P (shape {}):".format(P.shape))
# print(P)
# print("\nReward Matrix R (shape {}):".format(R.shape))
# print(R)


Iteration 0: Policy = [ 8 23 19 15  7]
Iteration 1: Policy = [16 14 26 14 19]
Iteration 2: Policy = [16 14 15 14  8]
Iteration 3: Policy = [16 14 15 25  8]

Optimal Policy: [16 14 15 25  8]
Value Function: [91.661 91.558 90.363 90.807 90.22 ]


In [2]:
import numpy as np
from scipy.optimize import linprog
from itertools import product

In [4]:
def generate_all_deterministic_policies(n_states, n_actions):
    """Generate all deterministic policies as one-hot matrices."""
    all_policies = []
    for actions in product(range(n_actions), repeat=n_states):
        policy = np.zeros((n_states, n_actions))
        for s, a in enumerate(actions):
            policy[s, a] = 1
        all_policies.append(policy)
    return all_policies

def decompose_random_policy(pi, deterministic_policies, seed=None):
    """
    Decompose a random policy into a convex combination of deterministic policies.
    You can randomize the LP objective to find different decompositions.
    """
    if seed is not None:
        np.random.seed(seed)
    
    n_states, n_actions = pi.shape
    N = len(deterministic_policies)
    
    # Flatten each deterministic policy
    P = np.array([policy.flatten() for policy in deterministic_policies]).T  # shape: (n_states * n_actions, N)
    b = pi.flatten()

    # Add equality constraints: sum_i λ_i P_i = pi, and sum_i λ_i = 1
    A_eq = np.vstack([P, np.ones(N)])
    b_eq = np.concatenate([b, [1]])

    # Use a random objective to get a different decomposition
    c = np.random.rand(N)

    res = linprog(c=c, A_eq=A_eq, b_eq=b_eq, bounds=[(0, 1)] * N, method="highs")

    if res.success:
        weights = res.x
        decomposition = [(w, deterministic_policies[i]) for i, w in enumerate(weights) if w > 1e-6]
        return decomposition
    else:
        return None

def print_decomposition(decomposition):
    for i, (weight, policy) in enumerate(decomposition):
        print(f"\nPolicy {i + 1}: weight = {weight:.4f}")
        print(policy.astype(int))

In [5]:
# --- Example Usage ---

n_states = 3
n_actions = 2

# Randomized policy: each row is a probability distribution over actions for a state
pi = np.array([
    [0.6, 0.4],
    [0.2, 0.8],
    [0.5, 0.5]
])

print("Target randomized policy:")
print(pi)

# Generate all deterministic policies
det_policies = generate_all_deterministic_policies(n_states, n_actions)

# Get two different decompositions using different seeds
print("\nFirst decomposition:")
decomp1 = decompose_random_policy(pi, det_policies, seed=0)
print_decomposition(decomp1)

print("\nSecond (different) decomposition:")
decomp2 = decompose_random_policy(pi, det_policies, seed=42)
print_decomposition(decomp2)


Target randomized policy:
[[0.6 0.4]
 [0.2 0.8]
 [0.5 0.5]]

First decomposition:

Policy 1: weight = 0.1000
[[1 0]
 [1 0]
 [1 0]]

Policy 2: weight = 0.5000
[[1 0]
 [0 1]
 [0 1]]

Policy 3: weight = 0.1000
[[0 1]
 [1 0]
 [1 0]]

Policy 4: weight = 0.3000
[[0 1]
 [0 1]
 [1 0]]

Second (different) decomposition:

Policy 1: weight = 0.1500
[[1 0]
 [1 0]
 [1 0]]

Policy 2: weight = 0.4500
[[1 0]
 [0 1]
 [0 1]]

Policy 3: weight = 0.0500
[[0 1]
 [1 0]
 [0 1]]

Policy 4: weight = 0.3500
[[0 1]
 [0 1]
 [1 0]]
